In [1]:
import requests
import pandas as pd
import re
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

In [2]:
# defining User Agents to avoid 403 status code
ua = UserAgent()
user_agent = ua.random

url = 'https://cfd.nu.edu.pk/all-departments/'  
headers = {'User-Agent': user_agent}

response = requests.get(url, headers=headers)
soup_cfd = BeautifulSoup(response.content, 'html.parser')


links = soup_cfd.find_all('a', string="Faculty")
link = [tag['href'] for tag in links]

In [3]:
cfd_id_ = []
cfd_name = []
cfd_designation = []
cfd_HEC_approved = []
cfd_education = []
cfd_email = []
cfd_department = []
cfd_extension = []
cfd_image_URL = []

In [ ]:
# Extracting faculty details
for dept_link in link:
    cfd_dept_req = requests.get(dept_link, headers=headers)
    soup_cfd_dept = BeautifulSoup(cfd_dept_req.content, 'html.parser')

    cfd_faculty = soup_cfd_dept.findAll('div', class_='unitech-teacher')

    for cfd_faculty_f in cfd_faculty:
        cfd_name.append(cfd_faculty_f.find('h4').text.strip())

        if cfd_faculty_f.find('p', class_="hec"):
            cfd_HEC_approved.append("True")
        else:
            cfd_HEC_approved.append("False")

        email_tag = cfd_faculty_f.find('p')
        cfd_email.append(email_tag.text.strip() if email_tag else "N/A")

        designation_tag = cfd_faculty_f.find('h6')
        cfd_designation.append(designation_tag.text.strip() if designation_tag else "N/A")

        profile_link_tag = cfd_faculty_f.find('a')
        profile_link = profile_link_tag['href'] if profile_link_tag else None

        if profile_link:
            cfd_dept_faculty_profile_req = requests.get(profile_link, headers=headers)
            soup_cfd_dept_faculty_profile_f = BeautifulSoup(cfd_dept_faculty_profile_req.content, 'html.parser')

            dept_tag = soup_cfd_dept_faculty_profile_f.find('ul', class_="teacher__address")
            cfd_department.append(dept_tag.find('li').text.strip() if dept_tag else "N/A")

            try:
                ext_text = dept_tag.find_all('li')[-1].text.strip().replace(" ", "").replace("\n", "")
                cfd_extension.append(ext_text if ext_text else "None")
            except:
                cfd_extension.append("None")

            img_tag = soup_cfd_dept_faculty_profile_f.find('div', class_="teacher__thumb").find('img')
            cfd_image_URL.append(img_tag['src'] if img_tag else "N/A")

            id_tag = soup_cfd_dept_faculty_profile_f.find('link', attrs={'rel': 'shortlink'})
            cfd_id_.append(id_tag['href'].split("=")[-1] if id_tag else "N/A")

            try:
                education_info = soup_cfd_dept_faculty_profile_f.findAll(
                    'div', class_="htc__skill__container progress__bar--2"
                )[0].find('ul').find('li').text.strip()
            except:
                try:
                    education_info = soup_cfd_dept_faculty_profile_f.findAll(
                        'div', class_="htc__skill__container progress__bar--2"
                    )[0].find('p').text.strip()
                except:
                    education_info = "N/A"
            cfd_education.append(education_info)

        else:
            cfd_department.append("N/A")
            cfd_extension.append("None")
            cfd_image_URL.append("N/A")
            cfd_id_.append("N/A")
            cfd_education.append("N/A")

min_length = min(len(cfd_name), len(cfd_HEC_approved), len(cfd_email), len(cfd_designation),
                 len(cfd_department), len(cfd_extension), len(cfd_image_URL), len(cfd_id_), len(cfd_education))

# had an error due to not being approved by HEC so had to assign minimum length for those datasets
cfd_name = cfd_name[:min_length]
cfd_HEC_approved = cfd_HEC_approved[:min_length]
cfd_email = cfd_email[:min_length]
cfd_designation = cfd_designation[:min_length]
cfd_department = cfd_department[:min_length]
cfd_extension = cfd_extension[:min_length]
cfd_image_URL = cfd_image_URL[:min_length]
cfd_id_ = cfd_id_[:min_length]
cfd_education = cfd_education[:min_length]



C:\Users\f-jaf\AppData\Local\Temp\ipykernel_4528\3194332683.py:6: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  cfd_faculty = soup_cfd_dept.findAll('div', class_='unitech-teacher')
C:\Users\f-jaf\AppData\Local\Temp\ipykernel_4528\3194332683.py:45: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  education_info = soup_cfd_dept_faculty_profile_f.findAll(
C:\Users\f-jaf\AppData\Local\Temp\ipykernel_4528\3194332683.py:50: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  education_info = soup_cfd_dept_faculty_profile_f.findAll(


In [5]:
df_cfd = pd.DataFrame({
    'ID': cfd_id_,
    'Name': cfd_name,
    'Designation': cfd_designation,
    'HEC Approved PHD Supervisor': cfd_HEC_approved,
    'Highest Education': cfd_education,
    'Email': cfd_email,
    'Department': cfd_department,
    'Extension': cfd_extension,
    'Image URL': cfd_image_URL
})


In [6]:
# writing to csv
df_cfd.to_csv('cfd.csv', index=False)

In [7]:
# looking at top 5 rows 
df_cfd.head()

,ID,Name,Designation,HEC Approved PHD Supervisor,Highest Education,Email,Department,Extension,Image URL
0,507202,Dr. Muhammad Fayyaz,Associate Professor & HOD,True,"Ph.D. (Computer Science), COMSATS University I...",m.fayyaz@nu.edu.pk,Department of Computer Science,Ext:160,https://cfd.nu.edu.pk/wp-content/uploads/2022/...
1,4748,Dr. Muhammad Shahzad,Professor,True,"Ph.D in Computing (RS & GIS) AIT, Thailand, 2012",shahzad.sarfraz@nu.edu.pk,Department of Computer Science,Ext:102,https://cfd.nu.edu.pk/wp-content/uploads/2019/...
2,6077,Dr. Muhammad Umar,Associate Professor,True,"PhD (Software Engineering), 2020",umar.aftab@nu.edu.pk,Department of Computer Science,Ext:198,https://cfd.nu.edu.pk/wp-content/uploads/2020/...
3,507956,Dr. Muhammad Usama,Associate Professor,False,"Post-Doc, Lahore University of Management Scie...",m.usama@nu.edu.pk,Department of Computer Science,Ext:311,https://cfd.nu.edu.pk/wp-content/uploads/2022/...
4,509433,Dr. Ammar Rafiq,Assistant Professor,False,N/A,ammar.rafiq@nu.edu.pk,Department of Computer Science,Ext:165,https://cfd.nu.edu.pk/wp-content/uploads/2025/...
